IMPORTS

In [2]:
import numpy as np

import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

CODE TO LOAD MAX-MIN POOLED MFCC FILES 

In [3]:
def load_mfcc_files(directory):
    mfcc_data = []
    for file in os.listdir(directory):
            print(file)
            file_path = os.path.join(directory, file)
            data = pd.read_csv(file_path, header=None).values
            mfcc_data.append(data)
            print(data.shape)
    return mfcc_data

# Load MFCC data
asha = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Asha Bhosle")
bhavgeet = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Bhavgeet")
kishor = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Kishor Kumar")
lavni = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Lavni")
michael = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Michael Jackson")
national = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\National Anthem")

# Debug: Print the number of files loaded from each directory
print(f"Asha Bhosle: {len(asha)} files")
print(f"Bhavgeet: {len(bhavgeet)} files")
print(f"Kishor Kumar: {len(kishor)} files")
print(f"Lavni: {len(lavni)} files")
print(f"Michael Jackson: {len(michael)} files")
print(f"National Anthem: {len(national)} files")

Aaiye Meharban - Howrah Bridge 1958 128 Kbps_MFCC.csv
(20, 25000)
Aaja Aaja - Yeh Raaste Hain Pyaar Ke 128 Kbps_MFCC.csv
(20, 25000)
Aao Huzoor Tumko - Kismet 128 Kbps_MFCC.csv
(20, 25000)
Aasmaa - Saand Ki Aankh 128 Kbps_MFCC.csv
(20, 25000)
Aise Jalta Hai Jiya - 1920 128 Kbps_MFCC.csv
(20, 25000)
Allahu - Dev 128 Kbps_MFCC.csv
(20, 25000)
Andhere - 31st October 128 Kbps_MFCC.csv
(20, 25000)
Ankh Milaoongi - Fiza 128 Kbps_MFCC.csv
(20, 25000)
Baila Baila - Khwahish 128 Kbps_MFCC.csv
(20, 25000)
Bhanwara Bada Nadan Asha Bhosle - Sahib Bibi Aur Ghulam 128 Kbps_MFCC.csv
(20, 25000)
bollywood_MKS 1978 - O Saathi Re Tere Bina-Female_MFCC.csv
(20, 25000)
bollywood_MS 1966 - Jhumka Gira Re_MFCC.csv
(20, 25000)
bollywood_Rangeela 1995 - Tanha Tanha Yahan Pe_MFCC.csv
(20, 25000)
bollywood_STK 1982 - Kitne Bhi Tu(Female)_MFCC.csv
(20, 25000)
bollywood_UJ 1981 - Dil Cheez Kya Hai_MFCC.csv
(20, 25000)
bollywood_UJ 1981 - In Aankhon Ki Masti Ke_MFCC.csv
(20, 25000)
Chain Aapko Mila - Footpath 128 

COMBINING DATA INTO A SINGLE NUMPY ARRAY WITH LABELS

In [4]:
#repeat songs

asha = np.array(asha)
bhavgeet = np.array(bhavgeet)
kishor=np.array(kishor)
lavni=np.array(lavni)
michael=np.array(michael)
national=np.array(national)
print(asha.shape)
print(bhavgeet.shape)
print(kishor.shape)
print(lavni.shape)
print(michael.shape)
print(national.shape)

asha=asha[:-1]
bhavgeet = bhavgeet[:-1]
kishor=kishor[:-1]
lavni=lavni[:-3]

michael=michael[:-1]
print("new")
print(asha.shape)
print(bhavgeet.shape)
print(kishor.shape)
print(lavni.shape)
print(michael.shape)
print(national.shape)


(50, 20, 25000)
(50, 20, 25000)
(50, 20, 25000)
(52, 20, 25000)
(50, 20, 25000)
(49, 20, 25000)
new
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)


In [5]:

# Combine data
data = np.concatenate((asha, bhavgeet, kishor, lavni, michael, national), axis=0)

# Create labels
labels = ['asha'] * len(asha) + ['bhavgeet'] * len(bhavgeet) + ['kishor'] * len(kishor) + ['lavni'] * len(lavni) + ['michael'] * len(michael) + ['national'] * len(national)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Combine data and labels into a single array
combined_data = np.empty((data.shape[0], data.shape[1], data.shape[2] + 1), dtype=object)
combined_data[:, :, :-1] = data
combined_data[:, 0, -1] = encoded_labels  # Add labels in the last column

# Check the shape of the combined data
print(f"Combined data shape: {combined_data.shape}")

#The shape of combined_data will be (294, 20, 25001), where the last element in the innermost dimension contains the encoded labels.

Combined data shape: (294, 20, 25001)


In [6]:
# Extract the data without the labels
X = combined_data[:, :, :-1]

# Extract the labels
y = combined_data[:, 0, -1]

# Print the shapes of X and y to verify
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
X = X.astype(np.float32)
y = y.astype(np.int32)
print(y)


X shape: (294, 20, 25000)
y shape: (294,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


label encoding

In [7]:

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y)

# Print the encoded labels to verify
print(f"Encoded labels: {encoded_y}")

Encoded labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


SPLITING TRAIN SET INTO TRAIN AND TEST

In [ ]:
"""import numpy as np
import pandas as pd
from category_encoders import TargetEncoder
from sklearn.model_selection import train_test_split

# Combine data
data = np.concatenate((asha, bhavgeet, kishor, lavni, michael, national), axis=0)

# Create labels
labels = ['asha'] * len(asha) + ['bhavgeet'] * len(bhavgeet) + ['kishor'] * len(kishor) + ['lavni'] * len(lavni) + ['michael'] * len(michael) + ['national'] * len(national)

# Convert data and labels to a DataFrame
df = pd.DataFrame(data.reshape(data.shape[0], -1))
df['label'] = labels

# Apply target encoding
target_encoder = TargetEncoder()
df['encoded_label'] = target_encoder.fit_transform(df[['label']], df['label'])

# Extract the data without the labels
X = df.drop(columns=['label', 'encoded_label']).values
y = df['encoded_label'].values

# Print the shapes of X and y to verify
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"x_train shape: {x_train.shape}")

# Reshape the training and testing data back to the original shape
x_train = x_train.reshape(-1, data.shape[1], data.shape[2])
x_test = x_test.reshape(-1, data.shape[1], data.shape[2])

print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")"""

AttributeError: 'numpy.ndarray' object has no attribute 'groupby'

In [8]:

# Step 1: Split the dataset into training and test sets (80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.2, random_state=42)

# Step 2: Split the training set into training and validation sets (80% training, 20% validation)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

print(x_train.shape)
print(x_test.shape)
print(x_val.shape)
print(x_val[:1])
print(x_val[:1].shape)


(176, 20, 25000)
(59, 20, 25000)
(59, 20, 25000)
[[[-469.76727 -469.76727 -469.76727 ...    0.         0.         0.     ]
  [   0.         0.         0.      ...    0.         0.         0.     ]
  [   0.         0.         0.      ...    0.         0.         0.     ]
  ...
  [   0.         0.         0.      ...    0.         0.         0.     ]
  [   0.         0.         0.      ...    0.         0.         0.     ]
  [   0.         0.         0.      ...    0.         0.         0.     ]]]
(1, 20, 25000)


MODEL shit

model trying multiple classifiers

THIS IS THE MODEL WHICH WORKS WELL!!!!!

In [24]:
import numpy as np
from tensorflow.keras import layers, models, callbacks
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming x_train, x_val, x_test, y_train, y_val, y_test are already defined
# and have the shape (176, 20, 25000)

# Build the CNN model for feature extraction
def build_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(32, 5, activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2, padding='same'))
    model.add(layers.Dropout(0.25))
    
    

    #model.add(layers.Conv1D(64, 3, activation='relu', padding='same'))
    #model.add(layers.BatchNormalization())
    #model.add(layers.MaxPooling1D(2, padding='same'))
    #model.add(layers.Dropout(0.25))
    
    model.add(layers.Conv1D(128, 5, activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling1D(2, padding='same'))
    model.add(layers.Dropout(0.25))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='softmax'))  # Assuming 6 classes for classification
    return model

input_shape = (20, 25000)  # Adjusted to match the input data shape
cnn_model = build_cnn_model(input_shape)

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the data to match the input shape of the model
x_train = x_train.reshape(-1, 20, 25000)
x_val = x_val.reshape(-1, 20, 25000)
x_test = x_test.reshape(-1, 20, 25000)

# Define early stopping callback
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
# Train the CNN model with early stopping
cnn_model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_val, y_val), callbacks=[early_stopping,lr_scheduler])


# Assuming X_test and y_test are your test data and labels
X_test = x_test.astype(np.float32)
y_test = y_test.astype(np.int32)  # Ensure y_test is integer type for sparse_categorical_crossentropy

# Evaluate the model on the test data
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test, batch_size=32)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")
cnn_model.summary()

# Predict the labels for the test set
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels if necessary

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print detailed classification report
print(classification_report(y_test, y_pred_classes))


c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.1591 - loss: 2.6277 - val_accuracy: 0.2034 - val_loss: 4.3961 - learning_rate: 0.0010
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.3087 - loss: 2.0032 - val_accuracy: 0.2712 - val_loss: 3.7133 - learning_rate: 0.0010
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.4282 - loss: 1.8393 - val_accuracy: 0.2034 - val_loss: 3.0448 - learning_rate: 0.0010
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.4688 - loss: 1.5289 - val_accuracy: 0.2034 - val_loss: 3.4948 - learning_rate: 0.0010
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.4736 - loss: 1.3493 - val_accuracy: 0.2034 - val_loss: 3.9553 - learning_rate: 0.0010
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.4388 - loss: 1.6506 - val_accuracy: 0.2203 - val_loss: 3.3495 - learning_rate: 0.0010
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.5712 - loss: 1.2846 - 

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_42 (Conv1D)              │ (None, 20, 32)         │     4,000,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 20, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_42 (MaxPooling1D) │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 10, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_43 (Conv1D)              │ (None, 10, 128)        │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_31          │ (None, 10, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_43 (MaxPooling1D) │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 5, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_15 (Flatten)            │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 128)            │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,312,692 (46.97 MB)

 Trainable params: 4,104,038 (15.66 MB)

 Non-trainable params: 576 (2.25 KB)

 Optimizer params: 8,208,078 (31.31 MB)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Accuracy: 0.7627118644067796
Precision: 0.7802585832246849
Recall: 0.7627118644067796
F1 Score: 0.7621434685841465
              precision    recall  f1-score   support

           0       0.73      0.73      0.73        11
           1       1.00      0.64      0.78        11
           2       0.83      1.00      0.91        10
           3       0.38      0.43      0.40         7
           4       0.62      0.62      0.62         8
           5       0.92      1.00      0.96        12

    accuracy                           0.76        59
   macro avg       0.75      0.74      0.73        59
weighted avg       0.78      0.76      0.76        59



In [ ]:

# Define the RBMLayer (assuming you have this implemented)
class RBMLayer(layers.Layer):
    def __init__(self, n_components):
        super(RBMLayer, self).__init__()
        self.n_components = n_components
        # Add your RBM layer initialization here

    def call(self, inputs):
        # Add your RBM layer logic here
        return inputs

def build_model(input_shape):
    model = models.Sequential()
    # Conv1D Layer: 64 filters, filter size of 5, stride of 1
    model.add(layers.Conv1D(64, 5, activation='relu', strides=1, input_shape=input_shape))
    model.add(BatchNormalization())
    # Pooling Layer: Pool size of 2
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(Dropout(0.3))
    
    model.add(layers.Flatten())
    
    # RBM Layer 1: 100 hidden units
    rbm_layer1 = RBMLayer(n_components=100)
    model.add(rbm_layer1)
    
    # RBM Layer 2: 50 hidden units
    rbm_layer2 = RBMLayer(n_components=50)
    model.add(rbm_layer2)
    
    # Add a dense layer for classification
    model.add(layers.Dense(6, activation='softmax'))  # Assuming 10 classes for classification
    
    return model

# Build the model
input_shape = (20, 25000)  # Example input shape
rbm_model = build_model(input_shape)

# Compile the model with a lower learning rate
optimizer = Adam(learning_rate=0.001)
rbm_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Assuming you have your training data in X_train and y_train
# And your validation data in X_val and y_val
history = rbm_model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=(x_val, y_val), callbacks=[early_stopping])


print("\nModel layers after initialization:")
for idx, layer in enumerate(rbm_model.layers):
    print(f"Layer {idx}: {layer.name}")
# Run a single prediction to initialize the model


NameError: name 'layers' is not defined

In [ ]:

from tensorflow.keras.models import Model

# Load your custom model
model = rbm_model

# Print the model summary to identify the layer names
model.summary()

# Choose the layer from which you want to extract features
layer_name = 'rbm_layer_57'

# Create a new model that outputs the desired layer
feature_extractor = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

# Prepare your input data
input_data = x_val[:1] # Your input data here

# Extract features
features = feature_extractor.predict(input_data)

print(features)

Model: "sequential_33"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_39 (Conv1D)              │ (None, 16, 64)         │     8,000,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_39 (MaxPooling1D) │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_33 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_56 (RBMLayer)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_57 (RBMLayer)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,016,096 (91.61 MB)

 Trainable params: 8,005,322 (30.54 MB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 16,010,646 (61.08 MB)

AttributeError: The layer sequential_33 has never been called and thus has no defined input.

In [80]:
# Ensure the model has been called with input data
rbm_model.predict(x_train[:1])  # Run a prediction with a single sample

# Define the function to extract features
def extract_features(model, layer_name, input_data):
    """
    Extract features from a specific layer of the model.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the layer to extract features from.
    input_data (numpy.ndarray): The input data.
    
    Returns:
    numpy.ndarray: The extracted features.
    """
    layer = model.get_layer(name=layer_name)
    feature_extractor = models.Model(inputs=model.input, outputs=layer.output)
    features = feature_extractor.predict(input_data)
    return features

# Example usage with the correct layer name
layer_name = 'rbm_layer_57'  # Replace with the actual name of the RBM layer
X_train_features = extract_features(rbm_model, layer_name, x_train)
X_test_features = extract_features(rbm_model, layer_name, x_test)

# Print the shape of the extracted features
print("X_train_features shape:", X_train_features.shape)
print("X_test_features shape:", X_test_features.shape)


# Print the shape of the extracted features
print("X_train_features shape:", X_train_features.shape)
print("X_test_features shape:", X_test_features.shape)

# Now you can use the extracted features for further processing, e.g., training a RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Train a RandomForestClassifier on the extracted features
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_features, y_train)

# Evaluate the classifier on the test features
test_accuracy = clf.score(X_test_features, y_test)
print(f"Test accuracy: {test_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


AttributeError: The layer sequential_33 has never been called and thus has no defined input.

In [74]:
rbm_model.predict(x_val[:1])

# Extract features from the RBM model
feature_extractor = models.Model(inputs=rbm_model.input, outputs=rbm_model.layers[-2].output)
x_train_features = feature_extractor.predict(x_train)
x_val_features = feature_extractor.predict(x_val)
x_test_features = feature_extractor.predict(x_test)

# Train additional classifiers
lr = LogisticRegression()
rf = RandomForestClassifier()
svc = SVC(probability=True)

lr.fit(x_train_features, y_train)
rf.fit(x_train_features, y_train)
svc.fit(x_train_features, y_train)
# Combine the classifiers using VotingClassifier
ensemble_model = VotingClassifier(estimators=[
    ('lr', lr),
    ('rf', rf),
    ('svc', svc)
], voting='soft')

ensemble_model.fit(x_train_features, y_train)

# Evaluate the ensemble model on the test data
y_pred = ensemble_model.predict(x_test_features)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

print(classification_report(y_test, y_pred))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


AttributeError: The layer sequential_32 has never been called and thus has no defined input.

In [73]:
# First, verify the model is properly initialized
print("Model summary before prediction:")
rbm_model.summary()

# Initialize the model with a single sample
print("\nInitializing model with first sample...")
initial_prediction = rbm_model.predict(x_val[:1])

# Verify the model layers
print("\nModel layers after initialization:")
for idx, layer in enumerate(rbm_model.layers):
    print(f"Layer {idx}: {layer.name}")

# Now create the feature extractor
feature_extractor = models.Model(
    inputs=rbm_model.input,
    outputs=rbm_model.get_layer(index=-2).output  # Get second-to-last layer explicitly
)

# Extract features
print("\nExtracting features...")
x_train_features = feature_extractor.predict(x_train)
x_val_features = feature_extractor.predict(x_val)
x_test_features = feature_extractor.predict(x_test)

# Continue with the ensemble model
lr = LogisticRegression()
rf = RandomForestClassifier()
svc = SVC(probability=True)

print("\nTraining classifiers...")
lr.fit(x_train_features, y_train)
rf.fit(x_train_features, y_train)
svc.fit(x_train_features, y_train)

ensemble_model = VotingClassifier(estimators=[
    ('lr', lr),
    ('rf', rf),
    ('svc', svc)
], voting='soft')

ensemble_model.fit(x_train_features, y_train)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

print(classification_report(y_test, y_pred))

Model summary before prediction:


Model: "sequential_32"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_38 (Conv1D)              │ (None, 16, 64)         │     8,000,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 16, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_38 (MaxPooling1D) │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_32 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_54 (RBMLayer)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_55 (RBMLayer)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,016,096 (91.61 MB)

 Trainable params: 8,005,322 (30.54 MB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 16,010,646 (61.08 MB)


Initializing model with first sample...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step

Model layers after initialization:
Layer 0: conv1d_38
Layer 1: batch_normalization_21
Layer 2: max_pooling1d_38
Layer 3: dropout_44
Layer 4: flatten_32
Layer 5: rbm_layer_54
Layer 6: rbm_layer_55
Layer 7: dense_53


AttributeError: The layer sequential_32 has never been called and thus has no defined input.

model with rbm which works 

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder

# Define the RBMLayer
class RBMLayer(layers.Layer):
    def __init__(self, n_components, learning_rate=0.01, k=10, **kwargs):
        super(RBMLayer, self).__init__(**kwargs)
        self.n_components = n_components
        self.learning_rate = learning_rate
        self.k = k  # Number of Gibbs sampling steps

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], self.n_components),
                                 initializer='random_normal',
                                 trainable=True)
        self.h_bias = self.add_weight(shape=(self.n_components,),
                                      initializer='zeros',
                                      trainable=True)
        self.v_bias = self.add_weight(shape=(input_shape[-1],),
                                      initializer='zeros',
                                      trainable=True)

    def call(self, inputs):
        # Positive phase
        h_prob = tf.nn.sigmoid(tf.matmul(inputs, self.W) + self.h_bias)
        h_state = tf.nn.relu(tf.sign(h_prob - tf.random.uniform(tf.shape(h_prob))))

        # Gibbs sampling
        for _ in range(self.k):
            v_prob = tf.nn.sigmoid(tf.matmul(h_state, tf.transpose(self.W)) + self.v_bias)
            v_state = tf.nn.relu(tf.sign(v_prob - tf.random.uniform(tf.shape(v_prob))))
            h_prob = tf.nn.sigmoid(tf.matmul(v_state, self.W) + self.h_bias)
            h_state = tf.nn.relu(tf.sign(h_prob - tf.random.uniform(tf.shape(h_prob))))

        # Negative phase
        v_prob = tf.nn.sigmoid(tf.matmul(h_state, tf.transpose(self.W)) + self.v_bias)

        # Update weights
        positive_grad = tf.matmul(tf.transpose(inputs), h_prob)
        negative_grad = tf.matmul(tf.transpose(v_prob), h_prob)

        self.W.assign_add(self.learning_rate * (positive_grad - negative_grad))
        self.v_bias.assign_add(self.learning_rate * tf.reduce_mean(inputs - v_prob, axis=0))
        self.h_bias.assign_add(self.learning_rate * tf.reduce_mean(h_prob - h_state, axis=0))

        return h_prob

def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv1D(64, 3, activation='relu', input_shape=input_shape, kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Conv1D(128, 3, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.Flatten())
    
    rbm_layer1 = RBMLayer(n_components=100)
    model.add(rbm_layer1)
    
    rbm_layer2 = RBMLayer(n_components=50)
    model.add(rbm_layer2)
    rbm_layer3 = RBMLayer(n_components=50)
    model.add(rbm_layer3)
    
    model.add(layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dense(6, activation='softmax'))  # Assuming 6 classes for classification
    return model

# Example usage
input_shape = (20, 25000)  # Adjusted to match the input data shape
model = build_model(input_shape)

# Ensure x_train and y_train are NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

# Check the shapes of x_train and y_train
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=16, validation_data=(x_val, y_val), callbacks=[early_stopping, reduce_lr])

c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



x_train shape: (176, 20, 25000)
y_train shape: (176,)
Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 7s 171ms/step - accuracy: 0.1935 - loss: 4.0846 - val_accuracy: 0.2034 - val_loss: 3.3268 - learning_rate: 0.0010
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - accuracy: 0.1296 - loss: 3.2671 - val_accuracy: 0.1356 - val_loss: 2.9455 - learning_rate: 0.0010
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.0975 - loss: 2.8932 - val_accuracy: 0.2034 - val_loss: 2.7211 - learning_rate: 0.0010
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 93ms/step - accuracy: 0.1759 - loss: 2.7019 - val_accuracy: 0.1356 - val_loss: 2.5920 - learning_rate: 0.0010
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 0.1723 - loss: 2.5257 - val_accuracy: 0.2034 - val_loss: 2.4384 - learning_rate: 0.0010
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.1003 - loss: 2.4322 - val_accuracy: 0.1356 - val_loss: 2.4002 - learning_rate: 0.0010
Epoch 7/100
11/11 ━━━━━━━━━━━━━━

model testing and summary

In [114]:


# Assuming X_test and y_test are your test data and labels
X_test = x_test.astype(np.float32)
y_test = y_test.astype(np.int32)  # Ensure y_test is integer type for sparse_categorical_crossentropy

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")
model.summary()



# Assuming you have your test data in X_test and y_test
# And your trained model is named 'model'

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels if necessary

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print detailed classification report
print(classification_report(y_test, y_pred_classes))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.1425 - loss: 1.8959 
Test accuracy: 0.1355932205915451


Model: "sequential_52"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_70 (Conv1D)              │ (None, 18, 64)         │     4,800,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_70 (MaxPooling1D) │ (None, 9, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_71 (Conv1D)              │ (None, 7, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_71 (MaxPooling1D) │ (None, 3, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_50 (Flatten)            │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_90 (RBMLayer)         │ (None, 100)            │        38,884 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_91 (RBMLayer)         │ (None, 50)             │         5,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_87 (Dense)                │ (None, 256)            │        13,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_88 (Dense)                │ (None, 6)              │         1,542 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,650,202 (55.89 MB)

 Trainable params: 4,883,400 (18.63 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 9,766,802 (37.26 MB)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
Accuracy: 0.13559322033898305
Precision: 0.018385521401896008
Recall: 0.13559322033898305
F1 Score: 0.03238047052871237
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        11
           1       0.00      0.00      0.00        11
           2       0.00      0.00      0.00        10
           3       0.00      0.00      0.00         7
           4       0.14      1.00      0.24         8
           5       0.00      0.00      0.00        12

    accuracy                           0.14        59
   macro avg       0.02      0.17      0.04        59
weighted avg       0.02      0.14      0.03        59



c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [34]:
# Define the function to extract features
def extract_features(model, layer_name, input_data):
    """
    Extract features from a specific layer of the model.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the layer to extract features from.
    input_data (numpy.ndarray): The input data.
    
    Returns:
    numpy.ndarray: The extracted features.
    """
    layer = model.get_layer(name=layer_name)
    feature_extractor = models.Model(inputs=model.input, outputs=layer.output)
    features = feature_extractor.predict(input_data)
    return features

# Example usage
layer_name = 'dense_12'  # Replace with the name of the layer you want to extract features from
X_train_features = extract_features(model, layer_name, x_train)
X_test_features = extract_features(model, layer_name, X_test)

# Print the shape of the extracted features
print("X_train_features shape:", X_train_features.shape)
print("X_test_features shape:", X_test_features.shape)

# Now you can use the extracted features for further processing, e.g., training a RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Train a RandomForestClassifier on the extracted features
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_features, y_train)

# Evaluate the classifier on the test features
test_accuracy = clf.score(X_test_features, y_test)
print(f"Test accuracy: {test_accuracy}")

AttributeError: The layer sequential_6 has never been called and thus has no defined input.

code to visualize what cdbn is doing (subject to change)

In [ ]:
def visualize_feature_maps(model, layer_name, input_data):
    """
    Visualize the feature maps of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    input_data (numpy.ndarray): The input data.
    """
    layer = model.get_layer(name=layer_name)
    feature_map_model = models.Model(inputs=model.input, outputs=layer.output)
    feature_maps = feature_map_model.predict(input_data)
    
    n_features = feature_maps.shape[-1]
    fig, axes = plt.subplots(1, n_features, figsize=(20, 5))
    
    for i in range(n_features):
        axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

def visualize_filters(model, layer_name):
    """
    Visualize the filters of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    """
    layer = model.get_layer(name=layer_name)
    filters, biases = layer.get_weights()
    
    # Normalize filter values to 0-1 so we can visualize them
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    
    n_filters = filters.shape[-1]
    fig, axes = plt.subplots(1, n_filters, figsize=(20, 5))
    
    for i in range(n_filters):
        f = filters[:, :, :, i]
        axes[i].imshow(f[:, :, 0], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

In [16]:
import numpy as np
from tensorflow.keras import layers, models
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming x_train, x_val, x_test, y_train, y_val, y_test are already defined
# and have the shape (176, 20, 25000)

# Build the CNN model for feature extraction
def build_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Flatten())
    return model

input_shape = (20, 25000, 1)  # Adjusted to match the input data shape
cnn_model = build_cnn_model(input_shape)

# Reshape the data to add the channel dimension
x_train = x_train.reshape(-1, 20, 25000, 1)
x_val = x_val.reshape(-1, 20, 25000, 1)
x_test = x_test.reshape(-1, 20, 25000, 1)


# Extract features using the CNN model
x_train_features = cnn_model.predict(x_train)
x_val_features = cnn_model.predict(x_val)
x_test_features = cnn_model.predict(x_test)

# Train a RandomForestClassifier on the extracted features
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train_features, y_train.ravel())

# Validate the model
y_val_pred = rf_classifier.predict(x_val_features)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

# Test the model
y_test_pred = rf_classifier.predict(x_test_features)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 754ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 668ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 697ms/step
Validation Accuracy: 0.6101694915254238
Test Accuracy: 0.6101694915254238


In [14]:
import numpy as np
from tensorflow.keras import layers, models
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming x_train, x_val, x_test, y_train, y_val, y_test are already defined
# and have the shape (176, 20, 25000)

# Build the CNN model for feature extraction
def build_cnn_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), padding='same'))
    model.add(layers.Flatten())
    return model

input_shape = (20, 25000, 1)  # Adjusted to match the input data shape
cnn_model = build_cnn_model(input_shape)

# Reshape the data to add the channel dimension
x_train = x_train.reshape(-1, 20, 25000, 1)
x_val = x_val.reshape(-1, 20, 25000, 1)
x_test = x_test.reshape(-1, 20, 25000, 1)

cnn_model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
cnn_model.fit(x_train,y_train,epochs=40, batch_size=16, validation_data=(x_val, y_val))
# Extract features using the CNN model
x_train_features = cnn_model.predict(x_train)
x_val_features = cnn_model.predict(x_val)
x_test_features = cnn_model.predict(x_test)

# Train a RandomForestClassifier on the extracted features
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(x_train_features, y_train.ravel())

# Validate the model
y_val_pred = rf_classifier.predict(x_val_features)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {val_accuracy}")

# Test the model
y_test_pred = rf_classifier.predict(x_test_features)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

Epoch 1/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 68s 6s/step - accuracy: 0.0102 - loss: 16.0391 - val_accuracy: 0.0339 - val_loss: 15.7962
Epoch 2/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - accuracy: 0.1204 - loss: 14.7456 - val_accuracy: 0.1525 - val_loss: 15.6711
Epoch 3/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - accuracy: 0.2016 - loss: 15.8679 - val_accuracy: 0.1525 - val_loss: 15.6588
Epoch 4/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 59s 5s/step - accuracy: 0.2429 - loss: 15.6973 - val_accuracy: 0.1525 - val_loss: 15.6586
Epoch 5/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.2065 - loss: 15.7634 - val_accuracy: 0.1525 - val_loss: 15.6585
Epoch 6/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.2020 - loss: 16.4892 - val_accuracy: 0.1525 - val_loss: 15.6585
Epoch 7/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 57s 5s/step - accuracy: 0.2296 - loss: 16.1814 - val_accuracy: 0.1525 - val_loss: 15.6585
Epoch 8/40
11/11 ━━━━━━━━━━━━━━━━━━━━ 58s 5s/step - accuracy: 0.2585 - loss: 15.8427 - val_accuracy: 0.1

KeyboardInterrupt: 

og model

In [1]:
# Define the RBMLayer (assuming you have this implemented)
class RBMLayer(layers.Layer):
    def __init__(self, n_components):
        super(RBMLayer, self).__init__()
        self.n_components = n_components
        # Add your RBM layer initialization here

    def call(self, inputs):
        # Add your RBM layer logic here
        return inputs

def build_model(input_shape):
    model = models.Sequential()
    # Conv1D Layer: 64 filters, filter size of 5, stride of 1
    model.add(layers.Conv1D(64, 5, activation='relu', strides=1, input_shape=input_shape))
    # Pooling Layer: Pool size of 2
    model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Flatten())
    
    # RBM Layer 1: 100 hidden units
    rbm_layer1 = RBMLayer(n_components=100)
    model.add(rbm_layer1)
    
    # RBM Layer 2: 50 hidden units
    rbm_layer2 = RBMLayer(n_components=50)
    model.add(rbm_layer2)
    
    # Fully Connected Layer: 128 units
    model.add(layers.Dense(128, activation='relu'))
    
    # Output Layer: Softmax with 6 units for each class
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

input_shape = (20, 25000)  # Adjusted to match the input data shape
model = build_model(input_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ensure x_train and y_train are NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Check the shapes of x_train and y_train
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

# Ensure x_train is a 3D NumPy array
# Train the model
model.fit(x_train, y_train, epochs=40, batch_size=32)

NameError: name 'layers' is not defined

In [23]:
# Assuming X_test and y_test are your test data and labels
X_test = x_test.astype(np.float32)
y_test = y_test.astype(np.int32)  # Ensure y_test is integer type for sparse_categorical_crossentropy

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")
model.summary()

# Predict the labels for the test set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert predictions to class labels if necessary

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print metrics
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Print detailed classification report
print(classification_report(y_test, y_pred_classes))


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4952 - loss: 1200.5188
Test accuracy: 0.508474588394165


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_4 (Conv1D)               │ (None, 16, 64)         │     8,000,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_6 (RBMLayer)          │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_7 (RBMLayer)          │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,199,508 (92.31 MB)

 Trainable params: 8,066,502 (30.77 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,133,006 (61.54 MB)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Accuracy: 0.5084745762711864
Precision: 0.4589004451292587
Recall: 0.5084745762711864
F1 Score: 0.47380128450989095
              precision    recall  f1-score   support

           0       0.56      0.45      0.50        11
           1       0.44      0.64      0.52        11
           2       0.00      0.00      0.00        10
           3       0.33      0.57      0.42         7
           4       0.36      0.50      0.42         8
           5       0.91      0.83      0.87        12

    accuracy                           0.51        59
   macro avg       0.43      0.50      0.46        59
weighted avg       0.46      0.51      0.47        59



c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` param

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Assuming X is your input data with shape (176, 20, 25000) and y is your labels with shape (176,)
X = np.random.rand(176, 20, 25000)  # Replace with your actual data
y = np.random.randint(0, 6, 176)    # Replace with your actual labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the data to fit the CNN input requirements
X_train = X_train.reshape(-1, 20, 25000, 1)
X_test = X_test.reshape(-1, 20, 25000, 1)

# Build the CNN model
cnn_model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(20, 25000, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu')
    
])

# Compile the CNN model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)
model.predict(x_train)
# Extract features using the trained CNN model
feature_extractor = tf.keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

# Train the Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_features, y_train)

# Predict and evaluate the model
y_pred = rf_classifier.predict(X_test_features)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Epoch 1/10
